Получение данных из файлов.


In [ ]:
from pandas.io.formats.style_render import DataFrame
import os.path

import numpy as np
import pandas as pd

def get_text_data(path, label = None):
    X = []
    for review in os.listdir(path):
        with open(os.path.join(path,review)) as f:
            rev = f.read()
            
        X.append(rev)
        
    y = [label]*len(X)
        
    return X, y

def to_data_frame():
  good_review = "good/"
  neutral_review = "neutral/"
  bad_review = "bad/"

  good_X, good_y = get_text_data(good_review, label = "good")
  neutral_X, neutral_y = get_text_data(neutral_review, label = "neutral")
  bad_X, bad_y = get_text_data(bad_review, label = "bad")

  X = np.concatenate((good_X, neutral_X, bad_X))
  y = np.concatenate((good_y, neutral_y, bad_y))
  data = np.column_stack((X, y))

  return pd.DataFrame(data, columns = ["review", "label"])

def get_size(path):
   maxSize = 0;
   for review in os.listdir(path):
     size = os.path.getsize(path + review)

     if (size > maxSize):
       maxSize = size

     #print(review + " - " + str(os.path.getsize(path + review)) + " бит")

   return maxSize
      
def get_sizes():
  print("Максимальная размерность нейтрального комментария - " + str(get_size("neutral/")) + " бит")
  print("Максимальная размерность хорошего комментария - " + str(get_size("good/")) + " бит")
  print("Максимальная размерность плохого комментария - " + str(get_size("bad/")) + " бит")

def save_dataset(dataset: DataFrame, path: str):
  dataset.to_pickle(path + ".pkl")

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

# сохранение набора данных
#dataset = to_data_frame()
#save_dataset(dataset, "dataset")

# получение и вывод на экран
#dataset = get_dataset("dataset")
#dataset.head()

#get_sizes()

Предварительная обработка данных:

In [ ]:
from typing import List

import pandas as pd
from pandas.io.formats.style_render import DataFrame

import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nlkt_stopwords = set(stopwords.words('russian'))

!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz
import spacy
sp = spacy.load('ru_core_news_sm')
spacy_stopwords = sp.Defaults.stop_words

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language="russian")

!pip install pymorphy2
import pymorphy2
morph_analyzer = pymorphy2.MorphAnalyzer()

from operator import itemgetter

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

def save_dataset(dataset: DataFrame, path: str):
  dataset.to_pickle(path + ".pkl")

def save_dataset_xlsx(dataset: DataFrame, path: str):
  dataset.to_excel(path + ".xlsx")

# метод возвращающий слова комментариев в виде списка, без символов и цифр
def get_words_without_symbols(review: str) -> List[str]:
  text_words = word_tokenize(review)
  review = [word.lower() for word in text_words if word.isalpha()]
  return review

def get_words_without_stop_words(review: List[str]) -> List[str]:
  review = [word for word in review if word not in nlkt_stopwords]
  review = [word for word in review if word not in spacy_stopwords]
  return review

def get_lemmed_words(review: List[str]) -> List[str]:
  review = [morph_analyzer.parse(word)[0].normal_form for word in review]
  return review

def get_stemmed_words(review: List[str]) -> List[str]:
  review = [stemmer.stem(word) for word in review]
  return review

def get_cleaned_words(review: str) -> List[str]:
  review = get_words_without_symbols(review)
  review = get_words_without_stop_words(review)
  review = get_lemmed_words(review)
  review = get_stemmed_words(review)
  review = [word for word in review if len(word) > 3]
  return review

def get_cleaned_reviews(reviews):
  cleaned_reviews = []
  for review in reviews:
    words = get_cleaned_words(review)
    cleaned_reviews.append(words)
  return cleaned_reviews

#dataset = get_dataset("dataset")
#reviews = dataset["review"]

#cleaned_reviews = get_cleaned_reviews(reviews)
#data = np.column_stack((cleaned_reviews, dataset["label"]))

#dataset = pd.DataFrame(data, columns = ["review", "label"])

#save_dataset(dataset, "cleaned_dataset")
#save_dataset_xlsx(dataset, "cleaned_dataset")

Получение словаря выборки:

In [ ]:
from typing import List

import pandas as pd
import pickle

from pandas.io.formats.style_render import DataFrame

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

def save_dataset(dataset: DataFrame, path: str):
  dataset.to_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

# получаем словарь выборки
def get_dictionary(reviews) -> List[str]:
  dictionary = {}

  for review in reviews:
    for word in review:
      if word in dictionary:
        dictionary[word] += 1
      else:
        dictionary[word] = 1

  dictionary = list(dictionary.items())

  for element in set(dictionary):
    if element[1] == 1:
        dictionary.remove(element)

  dictionary = sorted(set(dictionary), key=itemgetter(1), reverse = True)
  return dictionary

dictionary = get_dictionary(reviews)

words = []
counts = []
for element in dictionary:
  words.append(element[0])
  counts.append(element[1])

dictionary = set(dict(zip(words, counts)))

file = open('dictionary.bin', 'wb')
pickle.dump(dictionary, file)

print(dictionary)

Векторизация данных:

In [3]:
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from pandas.io.formats.style_render import DataFrame

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

newComments = []
for comment in reviews.tolist():
  newComments.append(str(comment))

tfidfconverter = TfidfVectorizer(max_features=5500, min_df=40, max_df=0.82)
X = tfidfconverter.fit_transform(newComments).toarray()

Случайный лес

In [56]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

from pandas.io.formats.style_render import DataFrame

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

reviews_string = []
for comment in reviews.tolist():
  reviews_string.append(str(comment))

def classification(max_feautures, min_df, max_df, n_estimators):
  vectorizer = TfidfVectorizer(max_features=max_feautures, min_df = min_df, max_df = max_df)
  reviews_vectorized = vectorizer.fit_transform(reviews_string).toarray()

  x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3, shuffle = True)

  classifier = RandomForestClassifier(n_estimators=n_estimators)

  classifier.fit(x_train, y_train) 
  y_pred = classifier.predict(x_test)

  print("Отчёт по классификации: ")
  print(classification_report(y_test, y_pred))
  print("Точность: " + str(accuracy_score(y_test, y_pred)))

  return accuracy_score(y_test, y_pred);

def test():
  accuracy_list = []
  value_list = []

  for value in range(100, 1000, 50):
    accuracy = classification(5500, 40, 0.82, value)
    accuracy_list.append(accuracy)
    value_list.append(value)

  data = np.column_stack((accuracy_list, value_list))
  df = pd.DataFrame(data, columns = ["Точность", "value"])
  print(df)

#classification(5500, 40, 0.82, 1000)
#test()

Опорные вектора

In [ ]:
import pandas as pd
import pickle

from pandas.io.formats.style_render import DataFrame

import scipy.sparse as sp
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

newComments = []
for comment in reviews.tolist():
  newComments.append(str(comment))

def classification(max_feautures, min_df, max_df, kernel, degree):
  vectorizer = TfidfVectorizer(max_features=max_feautures, min_df = min_df, max_df = max_df)
  reviews_vectorized = vectorizer.fit_transform(newComments).toarray()

  x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3, shuffle = True)

  classifier = SVC(kernel = kernel, degree = degree)

  classifier.fit(x_train, y_train) 
  y_pred = classifier.predict(x_test)

  print("Отчёт по классификации: ")
  print(classification_report(y_test, y_pred))
  print("Точность: " + str(accuracy_score(y_test, y_pred)))

  return accuracy_score(y_test, y_pred);

def test():
  accuracy_list = []
  value_list = []

  for value in range(1000, 11000, 500):
    accuracy = classification(5500, 40, 0.82, "poly", 2)
    accuracy_list.append(accuracy)
    value_list.append(value)

  data = np.column_stack((accuracy_list, value_list))
  df = pd.DataFrame(data, columns = ["Точность", "min_df"])
  print(df)

def get(max_feautures, min_df, max_df, kernel, degree, comment):
  reviews_string[0] = comment
  vectorizer = TfidfVectorizer(max_features=max_feautures, min_df = min_df, max_df = max_df)
  reviews_vectorized = vectorizer.fit_transform(reviews_string).toarray()

  x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3, shuffle = True)

  classifier = SVC(kernel = kernel, degree = degree)
  classifier.fit(x_train, y_train) 

  predicted = classifier.predict(x_test)


#classification(5500, 40, 0.82, "poly", 2)
#test()
#get(5500, 40, 0.82, "poly", 2, comment)

Соседи

In [ ]:
import pandas as pd
import pickle
import numpy as np

from pandas.io.formats.style_render import DataFrame

import scipy
from scipy import _lib

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
%matplotlib inline

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

newComments = []
for comment in reviews.tolist():
  newComments.append(str(comment))

def classification(max_feautures, min_df, max_df, n_neighbors):
  vectorizer = TfidfVectorizer(max_features=max_feautures, min_df = min_df, max_df = max_df)
  reviews_vectorized = vectorizer.fit_transform(newComments).toarray()

  x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3, shuffle = True)

  classifier = KNeighborsClassifier(n_neighbors = n_neighbors)

  classifier.fit(x_train, y_train) 
  y_pred = classifier.predict(x_test)

  print("Отчёт по классификации: ")
  print(classification_report(y_test, y_pred))
  print("Точность: " + str(accuracy_score(y_test, y_pred)))

  return accuracy_score(y_test, y_pred);

def test():
  accuracy_list = []
  value_list = []

  for value in range(1000, 11000, 500):
    accuracy = classification(value, 40, 0.82, 83)
    accuracy_list.append(accuracy)
    value_list.append(value)

  data = np.column_stack((accuracy_list, value_list))
  df = pd.DataFrame(data, columns = ["Точность", "max_feautures"])
  print(df)

def test_n():
  vectorizer = TfidfVectorizer(max_features=5500, min_df = 40, max_df = 0.82)
  reviews_vectorized = vectorizer.fit_transform(reviews_string).toarray()

  x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3, shuffle = True)

  error_rates = []
  for i in np.arange(1, 101):
    new_model = KNeighborsClassifier(n_neighbors = i)
    new_model.fit(x_train, y_train)
    new_predictions = new_model.predict(x_test)
    error_rates.append(np.mean(new_predictions != y_test))
  plt.plot(error_rates)

#classification(5500, 40, 0.82, 82)
#test_n()
#test()

Tensor MLP (multilayer perceptron)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout

import keras

from keras.preprocessing.text import Tokenizer
from pandas.io.formats.style_render import DataFrame
from sklearn.model_selection import train_test_split

import pandas as pd
import pickle
import numpy as np

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews.tolist())
textSequences = tokenizer.texts_to_sequences(reviews.tolist())

labels = dataset["label"]
labels[labels == "bad"] = 0
labels[labels == "neutral"] = 1
labels[labels == "good"] = 2

x_train, x_test, y_train, y_test = train_test_split(textSequences, labels, test_size=0.3, shuffle = True)

num_words = 10000

print(u'Преобразуем описания заявок в векторы чисел...')
tokenizer = Tokenizer(num_words=num_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Размерность X_train:', x_train.shape)
print('Размерность X_test:', x_test.shape)

print(u'Преобразуем категории в матрицу двоичных чисел '
      u'(для использования categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

epochs = 10

print(u'Собираем модель...')
model = Sequential()
model.add(Dense(512, input_shape=(num_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=epochs,
                    verbose=1,)

score = model.evaluate(x_test, y_test,
                       batch_size=32, verbose=1)
print()
print(u'Оценка теста: {}'.format(score[0]))
print(u'Оценка точности модели: {}'.format(score[1]))

Tensor LSTM (long short-term memory)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

import keras

from keras.preprocessing.text import Tokenizer
from pandas.io.formats.style_render import DataFrame
from sklearn.model_selection import train_test_split

import pandas as pd
import pickle
import numpy as np

def get_dataset(path: str) -> DataFrame:
  return pd.read_pickle(path + ".pkl")

dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews.tolist())
textSequences = tokenizer.texts_to_sequences(reviews.tolist())

labels = dataset["label"]
labels[labels == "bad"] = 0
labels[labels == "neutral"] = 1
labels[labels == "good"] = 2

x_train, x_test, y_train, y_test = train_test_split(textSequences, labels, test_size=0.3, shuffle = True)

num_words = 1000

print(u'Преобразуем описания заявок в векторы чисел...')
tokenizer = Tokenizer(num_words=num_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Размерность X_train:', x_train.shape)
print('Размерность X_test:', x_test.shape)

print(u'Преобразуем категории в матрицу двоичных чисел '
      u'(для использования categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

max_words = 0
for desc in reviews.tolist():
    words = len(desc)
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном описании заявки: {} слов'.format(max_words))

maxSequenceLength = max_words

print(u'Собираем модель...')
model = Sequential()
model.add(Embedding(1000, maxSequenceLength))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print (model.summary())

batch_size = 32
epochs = 3

print(u'Тренируем модель...')
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print()
print(u'Оценка теста: {}'.format(score[0]))
print(u'Оценка точности модели: {}'.format(score[1]))

Векторизация датасета и обучение модели

In [ ]:
dataset = get_dataset("cleaned_dataset")
reviews = dataset["review"]

reviewsToVectorizer = []
for review in reviews.tolist():
  reviewsToVectorizer.append(str(review))

vectorizer = TfidfVectorizer(max_features=5500, min_df = 40, max_df = 0.82)
reviews_vectorized = vectorizer.fit_transform(reviewsToVectorizer).toarray()

x_train, x_test, y_train, y_test = train_test_split(reviews_vectorized, dataset["label"], test_size=0.3)

classifier = SVC(kernel = "poly", degree = 2)

classifier.fit(x_train, y_train) 

Пользовательский ввод

In [88]:
input_review = input("Введите комментарий для оценки: ")
input_cleaned_review = get_cleaned_reviews([input_review])

input_vec = vectorizer.transform([str(input_cleaned_review)]).toarray()

predicted = classifier.predict(input_vec)

if (predicted[0] == "bad"):
  print("Эмоциональная оценка коментария - негативный.")
if (predicted[0] == "good"):
  print("Эмоциональная оценка коментария - позитивный.")
if (predicted[0] == "neutral"):
  print("Эмоциональная оценка коментария - нейтральный.")

Введите комментарий для оценки: Плохой
Эмоциональная оценка коментария - негативный.
